In [3]:

import cv2
from matplotlib import pyplot as plt
import numpy as np

#image = cv2.imread("samples/2_7mUR_original-test_1609981034838.jpg")
image = cv2.imread("../samples/IMG_20201127_001800.jpg")

def preprocess_image(img, grayscale=False, resize_shape=None, blur=True):
    """
    Preprocess image. This applies the following (some optional) steps:
    - Convert image to grayscale
    - Apply median blur to reduce noise

    Args:
        img: Image to preprocess
        grayscale: Convert image to grayscale if True
        resize_shape: Resize image to this shape if not None

    Returns:
        Preprocessed image
    """

    # Adjustable parameters
    median_blur_ksize = 7
    gaussian_blur_ksize = 7

    # Resize
    if resize_shape is not None:
        img = cv2.resize(img, resize_shape, interpolation=cv2.INTER_AREA)

    # Convert image to grayscale
    if grayscale:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    if blur:
        # Median blur
        img = cv2.medianBlur(img, median_blur_ksize)

        # gaussian blur
        #img = cv2.GaussianBlur(
         #   img, (gaussian_blur_ksize, gaussian_blur_ksize), 0)

    return img

image = preprocess_image(image, grayscale=False, resize_shape=(500, 500), blur=True)

cv2.imshow("Original", image)
cv2.waitKey(0)
cv2.destroyAllWindows()    


## Erosion

In [2]:
eroded = cv2.erode(image, np.ones((7,7), np.uint8), iterations=2)

mask = cv2.Canny(image, 40, 210)
"""
dilated = cv2.dilate(mask, None, iterations=1)
contours, hierarchy = cv2.findContours(
    dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)
# create a mask with the contours
mask = np.zeros_like(image)
cv2.drawContours(mask, contours, -1, (255), cv2.FILLED)
cv2.imshow("Mask", mask)
cv2.waitKey(0)
"""

 # Fill the holes
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
mask = cv2.Canny(gray, 40, 210)
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
filled_mask = np.zeros_like(mask)
cv2.drawContours(filled_mask, contours, -1, 255, thickness=cv2.FILLED) 
cv2.imshow("Filled Mask", filled_mask)
cv2.waitKey(0)


# Apply thresholding to obtain a binary image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, binary_image = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# switch zeros to ones
binary_image = cv2.bitwise_not(binary_image)

# Apply erosion to separate objects
kernel = np.ones((5, 5), np.uint8)
eroded_binary = cv2.erode(binary_image, kernel, iterations=2, anchor=(0, 0))

cv2.imshow("Eroded", eroded)
cv2.imshow("Binary", binary_image)
cv2.imshow("Binary Erorded", eroded_binary)
cv2.waitKey(0)
cv2.destroyAllWindows()